## Dynamically populate SQL query for Business view using parameters list, and Jinja template

In [0]:
parameters = [
    {
        "table" : "spotify_cata.silver.factstream",
        "alias" : "fs",
        "cols" : "fs.stream_id,fs.listen_duration"
    },
    {
        "table" : "spotify_cata.silver.dimuser",
        "alias" : "du",
        "cols" : "du.user_id,du.user_name",
        "condition" : "fs.user_id = du.user_id"
    },
    {
        "table" : "spotify_cata.silver.dimtrack",
        "alias" : "dt",
        "cols" : "dt.track_id,dt.track_name",
        "condition" : "fs.track_id = dt.track_id"
    }

]

In [0]:
pip install jinja2

In [0]:
from jinja2 import Template

In [0]:
query_txt = """
            SELECT 
                {% for param in parameters %}
                    {{ param.cols}}
                        {% if not loop.last %}
                            ,
                        {% endif %}
                {% endfor %}
            FROM
                {% for param in parameters %}
                    {% if loop.first %}
                        {{ param['table'] }} AS {{ param['alias'] }}
                    {% endif %}
                {% endfor %}
                {% for param in parameters %}
                    {% if not loop.first %}
                    LEFT JOIN
                        {{ param['table'] }} AS {{ param['alias'] }}
                    ON
                        {{ param['condition'] }}
                    {% endif %}
                {% endfor %}
"""

In [0]:
jinja_sql_str = Template(query_txt)
query = jinja_sql_str.render(parameters=parameters)
print(query)

In [0]:
display(spark.sql(query))